## Visualize periodic lightcurves

This notebooks demonstrates how to visualize lightcurves for periodic objects in the Rubin commissioning data.

In [ ]:
import astropy.units as u
import lsdb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from dask.distributed import Client
from io import StringIO
from nested_pandas import NestedDtype
from pathlib import Path

import warnings

# Disable plotting warnings
warnings.filterwarnings("ignore")

### The Rubin HATS catalogs

- The commissioning data in HATS is served from the LSDB shared directory for commissioning, at USDF.
- HATS (HEALPix Adaptive Tiling Scheme) is a HEALPix-based parquet format that enables fast parallelization of large scale workflows.
- The data was imported and post-processed by the [DASH pipeline](../dash_doc/README.md). 
- It includes added-value columns for scientific use - e.g. PSF and science magnitudes, and timestamps for every source / forced source.
- The latest data release available is that of weekly [2025_09](https://rubinobs.atlassian.net/browse/DM-49235).

In [ ]:
drp_release = "w_2025_09"
hats_dir = Path("/sdf/data/rubin/shared/lsdb_commissioning/hats")
hats_path = hats_dir / drp_release

### Load the Rubin data

The relevant catalogs are `diaObject_lc` and `object_lc`. Each row corresponds to a single object with all its sources and nested sources embedded in separate columns.

In [ ]:
def cast_nested(df, columns):
    return df.assign(
        **{
            col: df[col].astype(NestedDtype.from_pandas_arrow_dtype(df.dtypes[col]))
            for col in columns
        },
    )

In [ ]:
dia_catalog = lsdb.read_hats(hats_path / "diaObject_lc")
# We use the `cast_nested` utility method to cast columns into the NestedFrame type
dia_catalog = dia_catalog.map_partitions(
    cast_nested, columns=["diaSource", "diaForcedSource"]
)

In [ ]:
obj_catalog = lsdb.read_hats(hats_path / "object_lc")
# We use the `cast_nested` utility method to cast columns into the NestedFrame type
obj_catalog = obj_catalog.map_partitions(cast_nested, columns=["forcedSource"])

### Choose the variable objects

We have a selection of 10 variable objects, which were found from previous analysis of forced photometry on science images:

In [ ]:
variables_csv = """ra,dec,period
94.95546,-24.73952,0.12095
95.30235,-25.27431,0.12248
94.91626,-24.69648,0.12038
95.12418,-25.04329,0.23554
58.83506,-48.79122,0.56335
94.92264,-25.23185,0.07672
94.72086,-25.05767,0.17559
94.97073,-25.13643,0.12048
59.12997,-48.78522,0.11628
94.72086,-25.05767,0.17554
"""
# We add the index as a column of the DataFrame so we can later retrieve
# all the Rubin data from difference and science imaging
variables_df = pd.read_csv(StringIO(variables_csv)).reset_index()
# Transform the DataFrame into a LSDB Catalog
variables_catalog = lsdb.from_dataframe(variables_df)

### Crossmatch the data

Let's crossmatch the Rubin data with our desired variable objects:

In [ ]:
XMATCH_RADIUS_ARCSEC = 0.2
variable_object = variables_catalog.crossmatch(
    obj_catalog, radius_arcsec=XMATCH_RADIUS_ARCSEC, suffixes=["_var", ""]
)
variable_dia = variables_catalog.crossmatch(
    dia_catalog, radius_arcsec=XMATCH_RADIUS_ARCSEC, suffixes=["_var", ""]
)
# The result will have all the columns in obj_catalog suffixed with `_obj`,
# all the columns in dia_catalog suffixed with `_dia`, as well as all the
# columns in `variables_df` suffixed with `_var_obj`
result = variable_object.join(
    variable_dia, left_on="index_var", right_on="index_var", suffixes=["_obj", "_dia"]
)

And apply filtering according to the quality flags:

In [ ]:
for column in ["forcedSource_obj", "diaSource_dia", "diaForcedSource_dia"]:
    result = result.query(
        f"~{column}.psfFlux_flag"
        f" and ~{column}.pixelFlags_saturated"
        f" and ~{column}.pixelFlags_cr"
        f" and ~{column}.pixelFlags_bad"
    )

Notice that we instantiate a Dask Client to efficiently parallelize our computations:

In [ ]:
with Client(n_workers=20, threads_per_worker=1) as client:
    # Sort light curves by variable object index for plotting purposes
    result_df = result.compute().sort_values("index_var_obj")

### Plot the lightcurves from the Rubin Data

The following utility methods allow us to plot light curves from DIA source, DIA forced source and forcedSource, for each of our matches.

In [ ]:
COLORS = {
    "u": "#56b4e9",
    "g": "#009e73",
    "r": "#f0e442",
    "i": "#cc79a7",
    "z": "#d55e00",
    "y": "#0072b2",
}

#### In the magnitude scale

In [ ]:
def plot_mag_lightcurves(ax, row):
    """Plot magnitude light curves from DIA source, DIA forced source and forcedSource"""
    # Define flux types for each dataset
    datasets = [
        ("scienceFlux", "diaSourceTable_tract", row.diaSource_dia),
        ("psfDiffFlux", "forcedSourceOnDiaObjectTable", row.diaForcedSource_dia),
        ("psfFlux", "forcedSourceTable", row.forcedSource_obj),
    ]
    all_mags = [[], []]  # To store magnitudes for each row
    for i, (flux_col, table_name, lc) in enumerate(datasets):
        title = f"{flux_col} from {table_name}"
        flux_err_col = f"{flux_col}Err"
        ax[0, i].set_title(title)
        # Compute phase
        lc = lc.assign(
            phase=(lc.midpointMjdTai - lc.midpointMjdTai.loc[lc.psfFlux.idxmax()])
            % row.period_var_obj
            / row.period_var_obj
        )
        # First row: original light curve
        all_mags[0].extend(
            plot_mag_scale(
                ax[0, i],
                lc,
                flux_col,
                flux_err_col,
                x_name="midpointMjdTai",
                x_label="MJD",
                show_legend=(i == 0),
            )
        )
        # Second row: folded light curve
        all_mags[1].extend(
            plot_mag_scale(
                ax[1, i], lc, flux_col, flux_err_col, x_name="phase", x_label="Phase"
            )
        )
    return all_mags


def plot_mag_scale(ax, lc, flux_col, flux_err_col, x_name, x_label, show_legend=False):
    """Plot light curves in magnitude scale"""
    mag_values = []  # Store magnitudes for setting axis limits
    for band, color in COLORS.items():
        band_lc = lc.query(f"band == '{band}'")
        # Compute magnitudes and errors
        mag, magErr = create_mag_errors(band_lc[flux_col], band_lc[flux_err_col])
        ax.errorbar(
            band_lc[x_name],
            mag,
            magErr,
            fmt="o",
            label=band,
            color=color,
            alpha=1,
            markersize=5,
            capsize=3,
            elinewidth=1,
        )
        mag_values.extend(mag.dropna().values)  # Collect magnitude values
    ax.set_xlabel(x_label)
    ax.set_ylabel("Magnitude (AB)")
    ax.invert_yaxis()  # Magnitudes are plotted with brighter objects lower
    if show_legend:
        ax.legend(loc="lower right", fontsize=12)  # Show legend in top-left panel only
    return mag_values  # Return magnitudes for axis scaling


def create_mag_errors(sciFlux, sciFluxErr):
    """Move flux into magnitudes and calculate the error on the magnitude"""
    mag = u.nJy.to(u.ABmag, sciFlux)
    upper_mag = u.nJy.to(u.ABmag, sciFlux + sciFluxErr)
    lower_mag = u.nJy.to(u.ABmag, sciFlux - sciFluxErr)
    magErr = -(upper_mag - lower_mag) / 2
    return mag, magErr


def scale_mag_y_axis(ax, all_mags):
    """Set uniform y-axis scaling for each plot row"""
    for row_idx in range(2):
        if all_mags[row_idx]:  # Ensure we have data
            ymin, ymax = np.nanmin(all_mags[row_idx]), np.nanmax(all_mags[row_idx])
            for i in range(3):  # Apply limits to all columns in the row
                ax[row_idx, i].set_ylim(
                    ymax + 0.1, ymin - 0.1
                )  # Keep magnitude inverted

In [ ]:
for _, row in result_df.iterrows():
    fig, ax = plt.subplots(2, 3, figsize=(16, 8), sharex="row")  # 2 rows, 3 columns
    fig.suptitle(
        f"{drp_release} | RA={row.ra_var_obj:.5f}, Dec={row.dec_var_obj:.5f}",
        fontsize=16,
    )
    all_mags = plot_mag_lightcurves(ax, row)
    scale_mag_y_axis(ax, all_mags)
    plt.tight_layout()
    plt.show()

#### In the flux scale

In [ ]:
def plot_flux_lightcurves(ax, row):
    """Plot flux light curves from DIA source, DIA forced source and forcedSource"""
    # Define flux types for each dataset
    datasets = [
        ("scienceFlux", "diaSourceTable_tract", row.diaSource_dia),
        ("psfDiffFlux", "forcedSourceOnDiaObjectTable", row.diaForcedSource_dia),
        ("psfFlux", "forcedSourceTable", row.forcedSource_obj),
    ]
    all_flux = [[], [], []]  # To store flux for each row
    for i, (flux_col, table_name, lc) in enumerate(datasets):
        title = f"{flux_col} from {table_name}"
        flux_err_col = f"{flux_col}Err"
        ax[0, i].set_title(title)
        # Compute phase
        lc = lc.assign(
            phase=(lc.midpointMjdTai - lc.midpointMjdTai.loc[lc.psfFlux.idxmax()])
            % row.period_var_obj
            / row.period_var_obj
        )
        # First row: original light curve
        all_flux[i].extend(
            plot_flux_scale(
                ax[0, i],
                lc,
                flux_col,
                flux_err_col,
                x_name="midpointMjdTai",
                x_label="MJD",
                show_legend=(i == 0),
            )
        )
        # Second row: folded light curve
        all_flux[i].extend(
            plot_flux_scale(
                ax[1, i], lc, flux_col, flux_err_col, x_name="phase", x_label="Phase"
            )
        )
    return all_flux


def plot_flux_scale(ax, lc, flux_col, flux_err_col, x_name, x_label, show_legend=False):
    """Function to plot light curves in flux scale"""
    flux_values = []  # Store flux values for setting axis limits
    for band, color in COLORS.items():
        band_lc = lc.query(f"band == '{band}'")
        # Extract flux values and errors directly
        flux = band_lc[flux_col]
        flux_err = band_lc[flux_err_col]
        ax.errorbar(
            band_lc[x_name],
            flux,
            flux_err,
            fmt="o",
            label=band,
            color=color,
            alpha=1,
            markersize=5,
            capsize=3,
            elinewidth=1,
        )
        flux_values.extend(flux.dropna().values)  # Collect flux values
    ax.set_xlabel(x_label, fontsize=14)  # Increased font size
    ax.set_ylabel(f"{flux_col} (nJy)", fontsize=14)  # Increased font size
    if show_legend:
        ax.legend(loc="lower right", fontsize=12)  # Show legend in top-left panel only
    return flux_values  # Return flux values for axis scaling


def scale_flux_y_axis(ax, all_flux):
    """Set uniform y-axis scaling for first and third columns, while middle remains independent and symmetric"""
    for row_idx in range(2):
        for col_idx in [0, 2]:  # Apply shared limits to first and third columns
            if all_flux[col_idx]:  # Ensure we have data
                ymin, ymax = np.nanmin(all_flux[col_idx]), np.nanmax(all_flux[col_idx])
                ax[row_idx, col_idx].set_ylim(
                    ymin - 0.1 * abs(ymin), ymax + 0.1 * abs(ymax)
                )  # Add buffer
        # Middle column (psfDiffFlux) gets independent, symmetric scaling
        if all_flux[1]:
            max_abs_flux = np.nanmax(
                np.abs(all_flux[1])
            )  # Find the maximum absolute value
            ax[row_idx, 1].set_ylim(
                -max_abs_flux * 1.1, max_abs_flux * 1.1
            )  # Symmetric range around 0

In [ ]:
for _, row in result_df.iterrows():
    fig, ax = plt.subplots(2, 3, figsize=(16, 8), sharex="row")  # 2 rows, 3 columns
    fig.suptitle(
        f"{drp_release} | RA={row.ra_var_obj:.5f}, Dec={row.dec_var_obj:.5f}",
        fontsize=16,
    )
    all_flux = plot_flux_lightcurves(ax, row)
    scale_flux_y_axis(ax, all_flux)
    plt.tight_layout()
    plt.show()